<span style="font-size:3.0em">
    Wikipedia Scrape – Tony Nominees
    <br>
</span> 

<i> There's a lot here. All of it good! </i>

# Import Packages + Functions

## Get Links

In [1]:

from bs4 import BeautifulSoup
import requests
import urllib.request
import re

def getLinks_tagged_fast(url, tag):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    links = []
    # set the opening of each link to be...
    tag = tag
    for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
        links.append(link.get('href'))
    return links

## Get Wikipedia Stuff

In [2]:
import pandas as pd
from IPython.display import display_html
from bs4 import BeautifulSoup
import re

In [3]:
url = 'https://en.wikipedia.org/wiki/Tony_Award_for_Best_Musical'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc,'lxml')
#print(soup.prettify())

## Solve on Granular Level
_Solve without function_

In [125]:
table_classes = {"class": ["wikitable"]}
tables = soup.findAll("table", table_classes)

### Get Tables

In [126]:
table = tables[1]
parsed_table_data=[]
rows = table.findAll('tr')

### Set up Regex

In [127]:
# Use regex to extract year
# I keep the regex example for reference
string = '19504th Tony Awards'
re_1 = "^\d{4}"
re_2 = "\d{1,2}th Tony Awards"
yr = re.findall(re_1,string)[0]
string_2 = string.split(yr)[1]
t_yr = re.findall(re_2,string_2)[0]


### Use Regex on Rows

In [327]:
parsed_table_data = []
row_1 = rows[0]
children = row_1.findChildren(recursive=False)
r_n = len(children)

for row in rows:
    children = row.findChildren(recursive=False)
    
    row_text = []
    for child in children:
        
        clean_text = child.text
        
        # We can sort each row as follows:
        if re.findall(re_1,clean_text):
            string  = clean_text
            # get the year
            yr = re.findall(re_1,string)[0]
            # split the string
            string_2 = string.split(yr)[1]
            # get the tony year
            t_yr = re.findall(re_2,string_2)[0]
            
            # Send the cleaned text back to the loop
            clean_text = yr +' | ' + t_yr
            row_text.append(clean_text)
            continue

        #This is to discard reference/citation links
        clean_text = clean_text.split('&#91;')[0]
        #This is to clean the header row of the sort icons
        clean_text = clean_text.split('&#160;')[-1]
        # This discards demarcation of winner of pulitzer prize
        clean_text = clean_text.replace('†','')
        
        clean_text = clean_text.strip()
        row_text.append(clean_text)
    
    if '|' in row_text[0]:
        cell_1 = row_text[0]
        continue
    else:
        row_text.insert(0,cell_1)   
    
    if len(row_text) >r_n:
        row_text.pop(0)

    if len(row_text) <r_n:
        x = row_text[-1]
        # How many items are you missing? (diff)
        diff = r_n - len(row_text)
        for i in range(diff):
            # for each item missing, append
            row_text.append(x)
        
    parsed_table_data.append(row_text)

### Get Headers

In [328]:
# Get headers
headers = parsed_table_data.pop(0)
# Remove the first insertion
print(headers)

['Year', 'Musical', 'Book', 'Music', 'Lyrics']


### Create a Df

In [308]:
# create a df
df = pd.DataFrame(parsed_table_data, columns=headers)

### Random trick to check if a variable exists

In [370]:
if 'var_1' not in dir():
    var_1 = 3
    print('var_1 does not exist\nsetting var_1 = 3')
else:
    print('var_1 = {}'.format(var_1))

var_1 = 7


## Solve with Iterator

_Solve with a function_

In [7]:
def get_df_from_table(table_in):
    """
    returns a structured table for a beautiful soup table
    """
    
    ### SET UP REGEX
    
    re_1 = "^\d{4}"
    re_2 = "\d{1,2}(th|rd|st|nd) Tony Awards"


    # JUMP INTO IT!
    parsed_table_data=[]
    rows = table_in.findAll('tr')
    
    row_1 = rows[0]
    children = row_1.findChildren(recursive=False)
    r_n = len(children)

    parsed_table_data = []
    for row in rows:
        children = row.findChildren(recursive=False)

        row_text = []
        for child in children:

            clean_text = child.text

            # We can sort each row as follows:
            if re.findall(re_2,clean_text):
                string  = clean_text
                
                # get the year
                yr = re.match(re_1,string).group()
                # split the string
                string_2 = string.split(yr)[1]
                # get the tony year
                t_yr = re.match(re_2,string_2).group()
                
   

                # Send the cleaned text back to the loop
                clean_text = yr +' | ' + t_yr
                row_text.append(clean_text)
                
                continue

            #This is to discard reference/citation links
            clean_text = clean_text.split('&#91;')[0]
            #This is to clean the header row of the sort icons
            clean_text = clean_text.split('&#160;')[-1]
            # This discards demarcation of winner of pulitzer prize
            clean_text = clean_text.replace('†','')

            clean_text = clean_text.strip()
            row_text.append(clean_text)
            

        if '|' in row_text[0]:
            cell_1 = row_text[0]
            continue
            
        else:
            if 'cell_1' not in dir():
                cell_1 = 'nothing'
            row_text.insert(0, cell_1)

        if len(row_text) >r_n:
            # removes first item, we don't want it...
            row_text.pop(0)

        if len(row_text) <r_n:
            x = row_text[-1]
            # How many items are you missing? (diff)
            diff = r_n - len(row_text)
            for i in range(diff):
                # for each item missing, append
                row_text.append(x)

        parsed_table_data.append(row_text)
    
    ## ONTO PART II
    headers = parsed_table_data.pop(0)
    df = pd.DataFrame(parsed_table_data, columns=headers)
    return df

### Apply to List of Tables

In [8]:
table_classes = {"class": ["wikitable"]}
tables = soup.findAll("table", table_classes)

In [9]:
df=pd.DataFrame()
for table in tables:
    sub_df = get_df_from_table(table)
    df = df.append(sub_df,ignore_index=True)

## Save as a df

In [11]:
file_name = 'Wikipedia_scrape_Tony_nominees.csv'
df.to_csv(file_name)